In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
markables = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

markables.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,num_words,previous_words,next_words,is_singleton
0,1,[1],0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",0,0,1,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,2,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 10, 12, 13]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0]",1,0,3,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 10, 12, 13, 14, 15, 16]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 10, 12, 13, 14, 15, 16, 17, 18]","[0.0, 1.0]"
4,5,"[10, 12]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,2,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[13, 14, 15, 16, 17, 18, 4, 5, 19, 20]","[0.0, 1.0]"


In [60]:
# pairs_budi = pd.read_csv("data/training/mention_pairs_budi.csv")
pairs_soon = pd.read_csv("data/training/mention_pairs_soon.csv")
pairs_gilang = pd.read_csv("data/training/mention_pairs_gilang.csv")

pairs_soon.head()

,m1_id,m2_id,is_exact_match,is_words_match,is_substring,is_abbreviation,is_appositive,is_nearest_candidate,sentence_distance,word_distance,markable_distance,is_coreference
0,3,11,0,0,0,0,0,0,2,31,8,1
1,4,11,0,0,0,0,0,0,2,29,7,0
2,5,11,0,0,0,0,0,0,2,25,6,0
3,6,11,0,0,0,0,0,0,2,16,5,0
4,7,11,0,0,0,0,0,0,1,14,4,0


In [5]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

def get_data(markable_ids):
    indices = reduce(lambda a, b: a + [b], map(lambda a: markables.index[markables['id'] == a].tolist()[0], markable_ids), [])
    data = markables.loc[indices]
    
    data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
    data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
    data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
    data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

    data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
    is_singleton = np.vstack(data.is_singleton)
    
    return data_text, data_previous_words, data_next_words, data_syntactic, is_singleton

def get_pair_data(markable_ids_1, markable_ids_2):
    text_1, prev_1, next_1, syntactic_1, is_singleton_1 = get_data(markable_ids_1)
    text_2, prev_2, next_2, syntactic_2, is_singleton_2 = get_data(markable_ids_2)
    
    return text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, is_singleton_1, is_singleton_2

def get_relation_data(mention_pairs):
    return mention_pairs[['is_exact_match', 'is_words_match', 'is_substring', 'is_abbreviation', 'is_appositive', 'is_nearest_candidate', 'sentence_distance', 'word_distance', 'markable_distance']]

# Soon-Generated

In [6]:
text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, is_singleton_11, is_singleton_21 = get_pair_data(pairs_soon.m1_id, pairs_soon.m2_id)
relation_1 = get_relation_data(pairs_soon)
label_soon = np.vstack(to_categorical(pairs_soon.is_coreference, num_classes=2))

## Build Model

### Word

In [7]:
words_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix
)

In [8]:
words_model_1 = words_model_builder.create_model()

In [9]:
words_model_1.fit([text_11, text_21], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 8s 1ms/step - loss: 0.3057 - acc: 0.8836
Epoch 2/10
5548/5548 [==============================] - 5s 819us/step - loss: 0.2285 - acc: 0.8850
Epoch 3/10
5548/5548 [==============================] - 5s 855us/step - loss: 0.1967 - acc: 0.9009
Epoch 4/10
5548/5548 [==============================] - 5s 853us/step - loss: 0.1703 - acc: 0.9174
Epoch 5/10
5548/5548 [==============================] - 5s 848us/step - loss: 0.1646 - acc: 0.9182
Epoch 6/10
5548/5548 [==============================] - 5s 871us/step - loss: 0.1471 - acc: 0.9279
Epoch 7/10
5548/5548 [==============================] - 5s 856us/step - loss: 0.1351 - acc: 0.9346
Epoch 8/10
5548/5548 [==============================] - 5s 856us/step - loss: 0.1305 - acc: 0.9391
Epoch 9/10
5548/5548 [==============================] - 5s 907us/step - loss: 0.1286 - acc: 0.9400
Epoch 10/10
5548/5548 [==============================] - 5s 925us/step - loss: 0.1131 - acc: 0.9486


In [10]:
#words_model_1.save('models/coreference_classifiers/words_soon.model')

### Context

In [11]:
context_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix
)

In [12]:
context_model_1 = context_model_builder.create_model()

In [13]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 14s 3ms/step - loss: 0.3868 - acc: 0.8773
Epoch 2/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.3614 - acc: 0.8866
Epoch 3/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.3330 - acc: 0.8875
Epoch 4/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.3114 - acc: 0.8877
Epoch 5/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.2843 - acc: 0.8875
Epoch 6/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.2546 - acc: 0.8881
Epoch 7/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.2378 - acc: 0.8877
Epoch 8/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.2107 - acc: 0.8926
Epoch 9/10
5548/5548 [==============================] - 10s 2ms/step - loss: 0.2132 - acc: 0.9025
Epoch 10/10
5548/5548 [==============================] - 9s 2ms/step - loss: 0.1979 - acc: 0.9056


In [14]:
#context_model_1.save('models/coreference_classifiers/context_soon.model')

### Syntactic

In [15]:
syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [16]:
syntactic_model_1 = syntactic_model_builder.create_model()

In [17]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 4s 670us/step - loss: 0.3641 - acc: 0.8839
Epoch 2/10
5548/5548 [==============================] - 1s 212us/step - loss: 0.2560 - acc: 0.8854
Epoch 3/10
5548/5548 [==============================] - 1s 199us/step - loss: 0.2435 - acc: 0.8870
Epoch 4/10
5548/5548 [==============================] - 1s 199us/step - loss: 0.2315 - acc: 0.8873
Epoch 5/10
5548/5548 [==============================] - 1s 204us/step - loss: 0.2281 - acc: 0.8931
Epoch 6/10
5548/5548 [==============================] - 1s 207us/step - loss: 0.2247 - acc: 0.8974
Epoch 7/10
5548/5548 [==============================] - 1s 210us/step - loss: 0.2080 - acc: 0.9050
Epoch 8/10
5548/5548 [==============================] - 1s 196us/step - loss: 0.1863 - acc: 0.9207
Epoch 9/10
5548/5548 [==============================] - 1s 215us/step - loss: 0.1781 - acc: 0.9252
Epoch 10/10
5548/5548 [==============================] - 1s 212us/step - loss: 0.1669 - acc: 0.9286


In [24]:
#syntactic_model_1.save('models/coreference_classifiers/syntactic_soon.model')

### Words + Context

In [20]:
words_context_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix,
)

In [21]:
words_context_model_1 = words_context_model_builder.create_model()

In [22]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 21s 4ms/step - loss: 0.3397 - acc: 0.8691
Epoch 2/10
5548/5548 [==============================] - 12s 2ms/step - loss: 0.2331 - acc: 0.8877
Epoch 3/10
5548/5548 [==============================] - 13s 2ms/step - loss: 0.2006 - acc: 0.8962
Epoch 4/10
5548/5548 [==============================] - 13s 2ms/step - loss: 0.1929 - acc: 0.9039
Epoch 5/10
5548/5548 [==============================] - 13s 2ms/step - loss: 0.1752 - acc: 0.9164
Epoch 6/10
5548/5548 [==============================] - 13s 2ms/step - loss: 0.1526 - acc: 0.9286
Epoch 7/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.1395 - acc: 0.9340
Epoch 8/10
5548/5548 [==============================] - 12s 2ms/step - loss: 0.1357 - acc: 0.9387
Epoch 9/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.1159 - acc: 0.9497
Epoch 10/10
5548/5548 [==============================] - 12s 2ms/step - loss: 0.1082 - acc: 0.9539


In [26]:
#words_context_model_1.save('models/coreference_classifiers/words_context_soon.model')

### Words + Syntactic

In [44]:
words_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [30]:
words_syntactic_model_1 = words_syntactic_model_builder.create_model()

In [31]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.3572 - acc: 0.8612
Epoch 2/10
5548/5548 [==============================] - 5s 946us/step - loss: 0.2290 - acc: 0.8861
Epoch 3/10
5548/5548 [==============================] - 7s 1ms/step - loss: 0.1913 - acc: 0.9097
Epoch 4/10
5548/5548 [==============================] - 6s 1ms/step - loss: 0.1661 - acc: 0.9194
Epoch 5/10
5548/5548 [==============================] - 6s 1ms/step - loss: 0.1513 - acc: 0.9274
Epoch 6/10
5548/5548 [==============================] - 5s 983us/step - loss: 0.1394 - acc: 0.9402
Epoch 7/10
5548/5548 [==============================] - 6s 1ms/step - loss: 0.1269 - acc: 0.9466
Epoch 8/10
5548/5548 [==============================] - 6s 1ms/step - loss: 0.1150 - acc: 0.9506
Epoch 9/10
5548/5548 [==============================] - 6s 1ms/step - loss: 0.1104 - acc: 0.9557
Epoch 10/10
5548/5548 [==============================] - 6s 1ms/step - loss: 0.1080 - acc: 0.9578


In [33]:
#words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon.model')

### Context + Syntactic

In [36]:
context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [37]:
context_syntactic_model_1 = context_syntactic_model_builder.create_model()

In [38]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 18s 3ms/step - loss: 0.4378 - acc: 0.8717
Epoch 2/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.2795 - acc: 0.8868
Epoch 3/10
5548/5548 [==============================] - 10s 2ms/step - loss: 0.2373 - acc: 0.8877
Epoch 4/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.2257 - acc: 0.8881
Epoch 5/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.2136 - acc: 0.8877
Epoch 6/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.1952 - acc: 0.8866
Epoch 7/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.1903 - acc: 0.9043
Epoch 8/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.1696 - acc: 0.9198
Epoch 9/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.1629 - acc: 0.9268
Epoch 10/10
5548/5548 [==============================] - 11s 2ms/step - loss: 0.1529 - acc: 0.9360


In [40]:
#context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon.model')

In [43]:
words_context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [45]:
words_context_syntactic_model_1 = words_context_syntactic_model_builder.create_model()

In [58]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)

Epoch 1/10
5548/5548 [==============================] - 19s 3ms/step - loss: 0.0955 - acc: 0.9643
Epoch 2/10
5548/5548 [==============================] - 18s 3ms/step - loss: 0.0779 - acc: 0.9679
Epoch 3/10
5548/5548 [==============================] - 14s 3ms/step - loss: 0.0687 - acc: 0.9744
Epoch 4/10
5548/5548 [==============================] - 14s 3ms/step - loss: 0.0649 - acc: 0.9776
Epoch 5/10
5548/5548 [==============================] - 14s 3ms/step - loss: 0.0627 - acc: 0.9787
Epoch 6/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.0573 - acc: 0.9818
Epoch 7/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.0356 - acc: 0.9861
Epoch 8/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.0465 - acc: 0.9863
Epoch 9/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.0297 - acc: 0.9899
Epoch 10/10
5548/5548 [==============================] - 15s 3ms/step - loss: 0.0483 - acc: 0.9863


In [59]:
#words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon.model')

# Gilang-Generated

In [61]:
text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, is_singleton_12, is_singleton_22 = get_pair_data(pairs_gilang.m1_id, pairs_gilang.m2_id)
relation_2 = get_relation_data(pairs_gilang)
label_gilang = np.vstack(to_categorical(pairs_gilang.is_coreference, num_classes=2))

## Build Model

### Words

In [62]:
words_model_2 = words_model_builder.create_model()

In [63]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 15s 1ms/step - loss: 0.2556 - acc: 0.9249
Epoch 2/10
10473/10473 [==============================] - 10s 941us/step - loss: 0.1736 - acc: 0.9404
Epoch 3/10
10473/10473 [==============================] - 12s 1ms/step - loss: 0.1523 - acc: 0.9403
Epoch 4/10
10473/10473 [==============================] - 10s 968us/step - loss: 0.1363 - acc: 0.9405
Epoch 5/10
10473/10473 [==============================] - 10s 995us/step - loss: 0.1242 - acc: 0.9406
Epoch 6/10
10473/10473 [==============================] - 12s 1ms/step - loss: 0.1177 - acc: 0.9450
Epoch 7/10
10473/10473 [==============================] - 11s 1ms/step - loss: 0.1117 - acc: 0.9479
Epoch 8/10
10473/10473 [==============================] - 11s 1ms/step - loss: 0.1014 - acc: 0.9508
Epoch 9/10
10473/10473 [==============================] - 11s 1ms/step - loss: 0.1025 - acc: 0.9523
Epoch 10/10
10473/10473 [==============================] - 11s 1ms/step - loss: 0.0957 - acc: 

In [65]:
#words_model_2.save('models/coreference_classifiers/words_gilang.model')

### Context

In [66]:
context_model_2 = context_model_builder.create_model()

In [67]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 27s 3ms/step - loss: 0.2630 - acc: 0.9394
Epoch 2/10
10473/10473 [==============================] - 19s 2ms/step - loss: 0.2430 - acc: 0.9406
Epoch 3/10
10473/10473 [==============================] - 20s 2ms/step - loss: 0.2301 - acc: 0.9405
Epoch 4/10
10473/10473 [==============================] - 21s 2ms/step - loss: 0.2199 - acc: 0.9405
Epoch 5/10
10473/10473 [==============================] - 21s 2ms/step - loss: 0.2192 - acc: 0.9405
Epoch 6/10
10473/10473 [==============================] - 21s 2ms/step - loss: 0.2115 - acc: 0.9405
Epoch 7/10
10473/10473 [==============================] - 21s 2ms/step - loss: 0.2087 - acc: 0.9405
Epoch 8/10
10473/10473 [==============================] - 22s 2ms/step - loss: 0.2054 - acc: 0.9405
Epoch 9/10
10473/10473 [==============================] - 20s 2ms/step - loss: 0.1950 - acc: 0.9405
Epoch 10/10
10473/10473 [==============================] - 23s 2ms/step - loss: 0.1894 - acc: 0.9405

In [84]:
#context_model_2.save('models/coreference_classifiers/context_gilang.model')

### Syntactic

In [69]:
syntactic_model_2 = syntactic_model_builder.create_model()

In [70]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 10s 916us/step - loss: 0.2642 - acc: 0.9248
Epoch 2/10
10473/10473 [==============================] - 3s 305us/step - loss: 0.1755 - acc: 0.9400
Epoch 3/10
10473/10473 [==============================] - 3s 243us/step - loss: 0.1623 - acc: 0.9402
Epoch 4/10
10473/10473 [==============================] - 2s 230us/step - loss: 0.1489 - acc: 0.9412
Epoch 5/10
10473/10473 [==============================] - 3s 272us/step - loss: 0.1389 - acc: 0.9447
Epoch 6/10
10473/10473 [==============================] - 3s 308us/step - loss: 0.1279 - acc: 0.9503
Epoch 7/10
10473/10473 [==============================] - 3s 289us/step - loss: 0.1201 - acc: 0.9527
Epoch 8/10
10473/10473 [==============================] - 3s 324us/step - loss: 0.1119 - acc: 0.9546
Epoch 9/10
10473/10473 [==============================] - 3s 308us/step - loss: 0.1119 - acc: 0.9568
Epoch 10/10
10473/10473 [==============================] - 3s 293us/step - loss: 0.1078 - 

In [85]:
#syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang.model')

### Words + Context

In [72]:
words_context_model_2 = words_context_model_builder.create_model()

In [73]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 38s 4ms/step - loss: 0.2291 - acc: 0.9395
Epoch 2/10
10473/10473 [==============================] - 28s 3ms/step - loss: 0.1689 - acc: 0.9411
Epoch 3/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.1402 - acc: 0.9442
Epoch 4/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.1255 - acc: 0.9467
Epoch 5/10
10473/10473 [==============================] - 31s 3ms/step - loss: 0.1114 - acc: 0.9533
Epoch 6/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.1020 - acc: 0.9573
Epoch 7/10
10473/10473 [==============================] - 30s 3ms/step - loss: 0.0929 - acc: 0.9612
Epoch 8/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.0828 - acc: 0.9671
Epoch 9/10
10473/10473 [==============================] - 30s 3ms/step - loss: 0.0768 - acc: 0.9702
Epoch 10/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.0762 - acc: 0.9722

In [86]:
#words_context_model_2.save('models/coreference_classifiers/words_context_gilang.model')

### Words + Syntactic

In [75]:
words_syntactic_model_2 = words_syntactic_model_builder.create_model()

In [76]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 19s 2ms/step - loss: 0.3809 - acc: 0.8730
Epoch 2/10
10473/10473 [==============================] - 10s 927us/step - loss: 0.1526 - acc: 0.9396
Epoch 3/10
10473/10473 [==============================] - 10s 926us/step - loss: 0.1280 - acc: 0.9440
Epoch 4/10
10473/10473 [==============================] - 11s 1ms/step - loss: 0.1120 - acc: 0.9475
Epoch 5/10
10473/10473 [==============================] - 12s 1ms/step - loss: 0.1036 - acc: 0.9565
Epoch 6/10
10473/10473 [==============================] - 11s 1ms/step - loss: 0.0903 - acc: 0.9605
Epoch 7/10
10473/10473 [==============================] - 10s 964us/step - loss: 0.0893 - acc: 0.9630
Epoch 8/10
10473/10473 [==============================] - 13s 1ms/step - loss: 0.0803 - acc: 0.9627
Epoch 9/10
10473/10473 [==============================] - 12s 1ms/step - loss: 0.0779 - acc: 0.9658
Epoch 10/10
10473/10473 [==============================] - 13s 1ms/step - loss: 0.0769 - acc: 

In [87]:
#words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang.model')

### Context + Syntactic

In [78]:
context_syntactic_model_2 = context_syntactic_model_builder.create_model()

In [79]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 34s 3ms/step - loss: 0.3488 - acc: 0.9257
Epoch 2/10
10473/10473 [==============================] - 21s 2ms/step - loss: 0.1944 - acc: 0.9395
Epoch 3/10
10473/10473 [==============================] - 22s 2ms/step - loss: 0.1706 - acc: 0.9402
Epoch 4/10
10473/10473 [==============================] - 23s 2ms/step - loss: 0.1613 - acc: 0.9404
Epoch 5/10
10473/10473 [==============================] - 23s 2ms/step - loss: 0.1511 - acc: 0.9397
Epoch 6/10
10473/10473 [==============================] - 23s 2ms/step - loss: 0.1393 - acc: 0.9411
Epoch 7/10
10473/10473 [==============================] - 23s 2ms/step - loss: 0.1270 - acc: 0.9478
Epoch 8/10
10473/10473 [==============================] - 22s 2ms/step - loss: 0.1201 - acc: 0.9504
Epoch 9/10
10473/10473 [==============================] - 22s 2ms/step - loss: 0.1088 - acc: 0.9544
Epoch 10/10
10473/10473 [==============================] - 20s 2ms/step - loss: 0.1021 - acc: 0.9594

In [88]:
#context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang.model')

### Words + Context + Syntactic

In [81]:
words_context_syntactic_model_2 = words_context_syntactic_model_builder.create_model()

In [82]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)

Epoch 1/10
10473/10473 [==============================] - 41s 4ms/step - loss: 0.2433 - acc: 0.9290
Epoch 2/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.1641 - acc: 0.9400
Epoch 3/10
10473/10473 [==============================] - 30s 3ms/step - loss: 0.1291 - acc: 0.9402
Epoch 4/10
10473/10473 [==============================] - 30s 3ms/step - loss: 0.1174 - acc: 0.9415
Epoch 5/10
10473/10473 [==============================] - 30s 3ms/step - loss: 0.1130 - acc: 0.9486
Epoch 6/10
10473/10473 [==============================] - 31s 3ms/step - loss: 0.0952 - acc: 0.9578
Epoch 7/10
10473/10473 [==============================] - 31s 3ms/step - loss: 0.0819 - acc: 0.9648
Epoch 8/10
10473/10473 [==============================] - 31s 3ms/step - loss: 0.0779 - acc: 0.9672
Epoch 9/10
10473/10473 [==============================] - 31s 3ms/step - loss: 0.0662 - acc: 0.9705
Epoch 10/10
10473/10473 [==============================] - 29s 3ms/step - loss: 0.0633 - acc: 0.9734

In [89]:
#words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang.model')